# Load run data and convert to pandas dataframe

In [ ]:
# %matplotlib notebook

from wild_visual_navigation import WVN_ROOT_DIR
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

folder, model_name = "loss", False
folder, model_name = "classicial_learning", True
folder, model_name = "confidence_fn", False
folder, model_name = "feature", False
p = os.path.join(WVN_ROOT_DIR, f"scripts/ablations/{folder}_ablation/{folder}_ablation_test_results.pkl")

with open(p, "rb") as f:
    res = pickle.load(f)

In [ ]:
import scipy
import pandas as pd
import pickle
import copy

data = {
    "run_nr": [],
    "cfg_name": [],
    "train_scene": [],
    "test_scene": [],
    "test_auroc_gt": [],
    "test_auroc_prop": [],
    "test_auroc_anomaly_prop": [],
    "test_auroc_anomaly_gt": [],
}

for train_scene_name, scene_data in res.items():
    for config_path, config_data in scene_data.items():
        if model_name:
            name = config_path
        else:
            # its a path
            name = config_path.split("/")[-1][:-5]
        for run_nr, run_data in config_data.items():
            for test_scene_name, test_scene_data in run_data.items():
                data["cfg_name"].append(name)
                data["run_nr"].append(run_nr)
                data["train_scene"].append(train_scene_name)
                data["test_scene"].append(test_scene_name)
                data["test_auroc_gt"].append(test_scene_data["test_auroc_gt_image"])
                data["test_auroc_prop"].append(test_scene_data["test_auroc_proprioceptive_image"])
                data["test_auroc_anomaly_prop"].append(
                    test_scene_data.get("test_auroc_anomaly_proprioceptive_image", 0)
                )
                data["test_auroc_anomaly_gt"].append(test_scene_data.get("test_auroc_anomaly_gt_image", 0))

df = pd.DataFrame.from_dict(data)


data_new = {
    "cfg_name": [],
    "train_scene": [],
    "test_scene": [],
    "gt_mean": [],
    "prop_mean": [],
    "ano_prop_mean": [],
    "ano_gt_mean": [],
    "gt_std": [],
    "prop_std": [],
    "ano_prop_std": [],
    "ano_gt_std": [],
}
keys = []
for i in range(len(df)):
    n = data["cfg_name"][i]
    tr = data["train_scene"][i]
    te = data["test_scene"][i]

    a = df[(df["train_scene"] == tr) * (df["test_scene"] == te) * (df["cfg_name"] == n)]
    k = n + tr + te
    if k in keys:
        continue
    keys.append(k)
    a["test_auroc_gt"].mean()
    a["test_auroc_gt"].std()
    data_new["cfg_name"].append(n)
    data_new["train_scene"].append(tr)
    data_new["test_scene"].append(te)
    data_new["gt_mean"].append(a["test_auroc_gt"].mean())
    data_new["prop_mean"].append(a["test_auroc_prop"].mean())
    data_new["ano_prop_mean"].append(a["test_auroc_anomaly_prop"].mean())
    data_new["ano_gt_mean"].append(a["test_auroc_anomaly_gt"].mean())
    data_new["gt_std"].append(a["test_auroc_gt"].std())
    data_new["prop_std"].append(a["test_auroc_prop"].std())
    data_new["ano_prop_std"].append(a["test_auroc_anomaly_prop"].std())
    data_new["ano_gt_std"].append(a["test_auroc_anomaly_gt"].std())


df = pd.DataFrame.from_dict(data_new)
df = df.round(decimals=4)
df[df.select_dtypes(include=["number"]).columns] = df[df.select_dtypes(include=["number"]).columns] * 100
df = df.sort_values("gt_mean", ascending=False)
df[(df["train_scene"].str.find("forest") != -1)]

In [ ]:
df = df.sort_values("gt_mean", ascending=False)
scene = "forest"
df[(df["train_scene"].str.find(scene) != -1) * (df["test_scene"].str.find(scene) != -1)]

In [ ]:
df = df.sort_values("gt_mean", ascending=False)
scene = "grassland"
df[(df["train_scene"].str.find(scene) != -1) * (df["test_scene"].str.find(scene) != -1)]

In [ ]:
df = df.sort_values("gt_mean", ascending=False)
scene = "hilly"
df[(df["train_scene"].str.find(scene) != -1) * (df["test_scene"].str.find(scene) != -1)]

# Time Adaptation and Data Efficiency

In [ ]:
import pickle
from wild_visual_navigation import WVN_ROOT_DIR
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

with open(os.path.join(WVN_ROOT_DIR, "scripts/ablations/time_adaptation/time_adaptation_steps.pkl"), "rb") as f:
    res = pickle.load(f)

steps_div = 100
nr_training_data, nr_steps, nr_runs = 10, int(10000 / steps_div), 10

auroc_gt = {}
auroc_gt["forest"] = np.zeros((nr_training_data, nr_steps, nr_runs))
auroc_gt["grassland"] = np.zeros((nr_training_data, nr_steps, nr_runs))
auroc_gt["hilly"] = np.zeros((nr_training_data, nr_steps, nr_runs))
auroc_prop = {}
auroc_prop["forest"] = np.zeros((nr_training_data, nr_steps, nr_runs))
auroc_prop["grassland"] = np.zeros((nr_training_data, nr_steps, nr_runs))
auroc_prop["hilly"] = np.zeros((nr_training_data, nr_steps, nr_runs))


for data in res:
    percentage, steps, run = data["percentage"], data["steps"], data["run"]

    da = [v for v in data["results"].values()][0]

    auroc_gt[data["scene"]][int(percentage / 10) - 1, int(steps / steps_div), run] = da["test_auroc_gt_image"]
    auroc_prop[data["scene"]][int(percentage / 10) - 1, int(steps / steps_div), run] = da[
        "test_auroc_proprioceptive_image"
    ]

In [ ]:
import matplotlib.pyplot as plt


def plot(title, data):
    width_half_inch, width_double_inch = 88.9 / 25.4, 182.0 / 25.4
    height_inch = 60 / 25.4
    scale = 2
    fig, ax = plt.subplots(2, 1, figsize=(width_half_inch * scale, height_inch * scale), dpi=300)
    fig.set_tight_layout(True)
    plt.rcParams.update({"font.size": 16})
    ax[1].tick_params(axis="both", which="major", labelsize=12)

    # Plot Matrix
    label_y = [str(k) for k in range(10, 101, 10)]
    label_x = [str(200)] + [str(int(j / 1000)) + "k" for j in range(300, data.shape[1] * 102, 100)]

    im = ax[1].imshow(data, cmap=sns.color_palette("RdYlBu", as_cmap=True))
    ele = [0] + list(np.arange(8, 100, 10))

    ax[1].set_xticks(ele)
    ax[1].set_yticks(np.arange(len(label_y))[::2])

    index = np.array([0] + list(np.arange(8, 100, 10)))
    ax[1].set_xticklabels(np.array(label_x)[index])
    ax[1].set_yticklabels(label_y[::2])

    ax[1].invert_yaxis()
    ax[1].set_xlabel("Training Steps")
    ax[1].set_ylabel("Data Percentage")

    plt.setp(ax[1].get_xticklabels(), rotation=0, ha="center", rotation_mode="anchor")
    ax[1].set_aspect(3.5)

    # Plot color bar
    nr = 100
    bar = np.arange(0, nr)[None]
    im = ax[0].imshow(bar, cmap=sns.color_palette("RdYlBu", as_cmap=True))
    ax[0].set_aspect(2)
    ax[0].set_xticks(np.array([0, nr - 1]))
    ax[0].set_yticks(np.array([]))
    ax[0].set_xticklabels(np.array([round(data.min() * 100, 1), round(data.max() * 100, 1)]))
    ax[0].set_yticklabels([])

    ax[1].tick_params(axis="both", which="major", labelsize=12)
    ax[1].tick_params(axis="both", which="minor", labelsize=12)
    ax[0].tick_params(axis="both", which="major", labelsize=12)
    ax[0].tick_params(axis="both", which="minor", labelsize=12)

    ax[0].set_xlabel("AUROC in percentage")
    ax[0].xaxis.set_label_coords(0.5, -1)

    plt.subplots_adjust(hspace=-0.5)
    plt.show()
    fig.savefig("/tmp/img.png", dpi=300)


plot("Time/Data-Adaptation AUCROC GT", auroc_prop["forest"][:-1, 1:].mean(axis=2))
plot("Time/Data-Adaptation AUCROC GT", auroc_gt["forest"][:-1, 1:].mean(axis=2))
plot("Time/Data-Adaptation AUCROC GT", auroc_prop["hilly"][:-1, 1:].mean(axis=2))
plot("Time/Data-Adaptation AUCROC GT", auroc_gt["hilly"][:-1, 1:].mean(axis=2))
plot("Time/Data-Adaptation AUCROC GT", auroc_prop["grassland"][:-1, 1:].mean(axis=2))
plot("Time/Data-Adaptation AUCROC GT", auroc_gt["grassland"][:-1, 1:].mean(axis=2))